In [0]:
%pip install faker


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from faker import Faker
import pandas as pd

fake = Faker()
data = [{
    "Name": fake.name(),
    "Address": fake.address(),
    "Email": fake.email()
} for _ in range(10)]

df = pd.DataFrame(data)
df_spark = spark.createDataFrame(df)
df_spark.show()


+--------------------+--------------------+--------------------+
|                Name|             Address|               Email|
+--------------------+--------------------+--------------------+
|         Donna Brown|195 Russell Viadu...|jillramirez@examp...|
|       Dennis Torres|674 Robert Orchar...|danieldiaz@exampl...|
|       Felicia Lynch|210 Stefanie Stre...|crossannette@exam...|
|       Nicole Spears|6715 Johnson Run ...|xwilliams@example...|
|   Alyssa Collins MD|0898 Johnson Loop...|deborah59@example...|
|         Brian Hogan|844 Madison Vista...|ortegajames@examp...|
|  Charles Fitzgerald|227 Stephen Stati...|mcleancharles@exa...|
|Christopher Petersen|9612 Benjamin Lak...|timothyalvarez@ex...|
|    Justin Nicholson|87463 Fisher Burg...|   cwolf@example.net|
|     Miss Erin Allen|598 Gregory Loaf\...| xdeleon@example.net|
+--------------------+--------------------+--------------------+



In [0]:
delta_path = "/tmp/fake_data_delta"


In [0]:
# Save as a Delta Table (managed table)
df_spark.write.format("delta").mode("overwrite").saveAsTable("fake_data_delta")


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "fake_data_delta")
delta_table.toDF().show(truncate=False)


+--------------------+------------------------------------------------------+--------------------------+------------------+
|Name                |Address                                               |Email                     |Inserted_Timestamp|
+--------------------+------------------------------------------------------+--------------------------+------------------+
|Donna Brown         |195 Russell Viaduct Suite 471\nKellyfurt, NE 27617    |jillramirez@example.com   |NULL              |
|Dennis Torres       |674 Robert Orchard\nLake Jason, CO 07659              |danieldiaz@example.net    |NULL              |
|Felicia Lynch       |210 Stefanie Street Apt. 518\nLake Ebonyview, LA 41591|crossannette@example.org  |NULL              |
|Nicole Spears       |6715 Johnson Run Apt. 664\nWest Barbaraport, CO 49460 |xwilliams@example.com     |NULL              |
|Alyssa Collins MD   |0898 Johnson Loop\nWilliamport, MA 14131              |deborah59@example.net     |NULL              |
|Brian H

In [0]:
df_spark.write.format("delta").mode("append").saveAsTable("fake_data_delta")


In [0]:
from delta.tables import DeltaTable

# Load the Delta Table
delta_table = DeltaTable.forName(spark, "fake_data_delta")

# Show latest contents
delta_table.toDF().show(truncate=False)


+--------------------+------------------------------------------------------+--------------------------+------------------+
|Name                |Address                                               |Email                     |Inserted_Timestamp|
+--------------------+------------------------------------------------------+--------------------------+------------------+
|Donna Brown         |195 Russell Viaduct Suite 471\nKellyfurt, NE 27617    |jillramirez@example.com   |NULL              |
|Dennis Torres       |674 Robert Orchard\nLake Jason, CO 07659              |danieldiaz@example.net    |NULL              |
|Felicia Lynch       |210 Stefanie Street Apt. 518\nLake Ebonyview, LA 41591|crossannette@example.org  |NULL              |
|Nicole Spears       |6715 Johnson Run Apt. 664\nWest Barbaraport, CO 49460 |xwilliams@example.com     |NULL              |
|Alyssa Collins MD   |0898 Johnson Loop\nWilliamport, MA 14131              |deborah59@example.net     |NULL              |
|Brian H

In [0]:
from faker import Faker
import pandas as pd
from datetime import datetime

fake = Faker()

def append_fake_data(n_rows=5):
    data = [{
        "Name": fake.name(),
        "Address": fake.address(),
        "Email": fake.email(),
        "Inserted_Timestamp": datetime.now().isoformat()
    } for _ in range(n_rows)]

    df = pd.DataFrame(data)
    df_spark = spark.createDataFrame(df)
    
    # Append to Delta Table
    df_spark.write.format("delta").mode("append").saveAsTable("fake_data_delta")
    print(f"{n_rows} fake rows appended at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [0]:
%sql
ALTER TABLE fake_data_delta ADD COLUMNS (Inserted_Timestamp STRING)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6367595720197352>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'ALTER TABLE fake_data_delta ADD COLUMNS (Inserted_Timestamp STRING)\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:147, in SqlMagic.sql(self, line, cell)
    143     raise Exception(
    144         

In [0]:
# Show current Delta table schema
delta_table.toDF().printSchema()


root
 |-- Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Inserted_Timestamp: string (nullable = true)



In [0]:
append_fake_data(5)


5 fake rows appended at 2025-07-10 08:37:16


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "fake_data_delta")
delta_table.toDF().orderBy("Inserted_Timestamp", ascending=False).show(truncate=False)


+---------------------+--------------------------------------------------------+--------------------------+--------------------------+
|Name                 |Address                                                 |Email                     |Inserted_Timestamp        |
+---------------------+--------------------------------------------------------+--------------------------+--------------------------+
|Cody Bright          |11264 Levine Garden Apt. 704\nNorth Richard, WI 36611   |jamesjohnson@example.com  |2025-07-10T08:37:14.985870|
|David Schneider      |7628 Eric Freeway\nEast Davidshire, PA 83364            |nicole96@example.net      |2025-07-10T08:37:14.985488|
|Anthony Craig        |Unit 5425 Box 2284\nDPO AE 43837                        |andersonjeremy@example.net|2025-07-10T08:37:14.985187|
|Catherine Combs      |44971 Monica Extension Suite 960\nMichaelshire, MS 11583|palmermichael@example.org |2025-07-10T08:37:14.984903|
|Barry Johnson        |9821 Campbell Greens Apt. 520\nK

In [0]:
spark.conf.set("spark.sql.session.timeZone", "Asia/Kolkata")


In [0]:
delta_table.history().select("version", "timestamp", "operation").show(truncate=False)


+-------+-------------------+---------------------------------+
|version|timestamp          |operation                        |
+-------+-------------------+---------------------------------+
|24     |2025-07-10 14:07:17|WRITE                            |
|23     |2025-07-10 14:06:56|WRITE                            |
|22     |2025-07-10 14:06:54|CREATE OR REPLACE TABLE AS SELECT|
|21     |2025-07-10 14:06:06|WRITE                            |
|20     |2025-07-10 14:06:03|CREATE OR REPLACE TABLE AS SELECT|
|19     |2025-07-10 14:04:43|WRITE                            |
|18     |2025-07-10 14:04:38|CREATE OR REPLACE TABLE AS SELECT|
|17     |2025-07-10 14:00:14|WRITE                            |
|16     |2025-07-10 14:00:11|CREATE OR REPLACE TABLE AS SELECT|
|15     |2025-07-10 13:59:21|WRITE                            |
|14     |2025-07-10 13:59:19|CREATE OR REPLACE TABLE AS SELECT|
|13     |2025-07-10 13:52:45|WRITE                            |
|12     |2025-07-10 13:52:42|CREATE OR R

In [0]:
spark.read.format("delta").option("versionAsOf", 0).table("fake_data_delta").show(truncate=False)


+----------------+----------------------------------------------------------+---------------------------+
|Name            |Address                                                   |Email                      |
+----------------+----------------------------------------------------------+---------------------------+
|Alexis Carter   |9190 King Port\nKatherineshire, OH 65503                  |hector19@example.com       |
|Jonathan Kemp   |04742 Danielle Field\nLake Shelleyfurt, MT 18255          |idaugherty@example.net     |
|Frank Mitchell  |848 Steve Views\nEast Brianfurt, NV 57799                 |dwright@example.org        |
|Ricky Saunders  |3724 Hatfield Prairie\nPort Michael, OH 71249             |kara96@example.org         |
|Eddie Velazquez |1960 Kimberly Common Suite 317\nLake Shannonfurt, VI 87310|pporter@example.org        |
|Margaret Webster|238 Joshua Vista\nSolismouth, WV 98324                    |scott71@example.com        |
|Tammy Jackson   |8376 Bender Bridge\nPort Kri

In [0]:
delta_table.toDF().orderBy("Inserted_Timestamp", ascending=False).show(truncate=False)


+---------------------+--------------------------------------------------------+--------------------------+--------------------------+
|Name                 |Address                                                 |Email                     |Inserted_Timestamp        |
+---------------------+--------------------------------------------------------+--------------------------+--------------------------+
|Cody Bright          |11264 Levine Garden Apt. 704\nNorth Richard, WI 36611   |jamesjohnson@example.com  |2025-07-10T08:37:14.985870|
|David Schneider      |7628 Eric Freeway\nEast Davidshire, PA 83364            |nicole96@example.net      |2025-07-10T08:37:14.985488|
|Anthony Craig        |Unit 5425 Box 2284\nDPO AE 43837                        |andersonjeremy@example.net|2025-07-10T08:37:14.985187|
|Catherine Combs      |44971 Monica Extension Suite 960\nMichaelshire, MS 11583|palmermichael@example.org |2025-07-10T08:37:14.984903|
|Barry Johnson        |9821 Campbell Greens Apt. 520\nK

In [0]:
# Step 1: Email config
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

sender_email = "lovisha1818@gmail.com"
receiver_email = "lovisha1818@gmail.com"
app_password = "yjnw tolo xwsq cptl"  # Gmail App Password

# Step 2: Email send function
def send_email(subject, html_table):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    body = f"""
    <html>
      <body>
        <h3>New Data Appended to Delta Table</h3>
        {html_table}
      </body>
    </html>
    """
    msg.attach(MIMEText(body, 'html'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, app_password)
            server.send_message(msg)
        print("✅ Email sent!")
    except Exception as e:
        print("❌ Email failed:", e)

# Step 3: Get last 5 rows added & send email
new_rows = delta_table.toDF().orderBy("Inserted_Timestamp", ascending=False).limit(5)
html_table = new_rows.toPandas().to_html(index=False, escape=False)
send_email("Delta Append Notification", html_table)


✅ Email sent!


In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forName(spark, "fake_data_delta")



In [0]:
# Show Delta Table version history
delta_table.history().show()


+-------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|        userId|            userName|           operation| operationParameters|                 job|notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     40|2025-07-10 14:36:04|70408274139286|lovisha1818@gmail...|               WRITE|{mode -> Append, ...|{572834352991046,...|    NULL|0710-081108-97s75...|         39|WriteSerializable|         true|{numFiles -> 1, n...|   

In [0]:
spark.conf.set("spark.sql.session.timeZone", "Asia/Kolkata")
